In [ ]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pandas as pd

# Exercise

Go to http://books.toscrape.com/, Using what you have learned create a csv file the contains all the books found in the website. The csv file should contain the following:

- Title
- Price
- Description
- Availability

Code guides have been provided to help you in creating the web scraper. Found below is the `get_title_links_and_next_page` this function returns 2 things book urls in a page and link to the next page. The idea here is to collect first all the book links available in the website and store the links in the `title_links` variable **(5 points)**

In [ ]:
base_url = "http://books.toscrape.com/"

def get_title_links_and_next_page(page_url):
    list_links = [] 
    page = urlopen(base_url)
    soup = BeautifulSoup(page, 'html.parser')
    for article in soup.find_all('article'):
        if "catalogue" not in article.find("a")['href']:
            url = base_url + "catalogue/" + article.find("a")['href']
        else:
            url = base_url + article.find("a")['href']
        list_links.append(url)
    
    try:
        next_url = base_url +  li["next"].find("a")["href"]
    except:
        next_url = None

    return (list_links, next_url)

res = get_title_links_and_next_page('http://books.toscrape.com/index.html')
title_links = res[0]

while res[1]:
    if "catalogue" not in res[1]:
        page_url = base_url + "catalogue/" + res[1]
    else:
        page_url = base_url + res[1]
    res = get_title_links_and_next_page(page_url)
    title_links += list_links

title_links

Once you have a list of all the available book links we loop through the links and use the 4 functions `get_title`, `get_price`, `get_description`, `get_availability` to retrieve the book information. **(10 points)**

In [ ]:
def get_title(soup):
    return soup.find("h1")

def get_price(soup):
    return soup.find("p", attrs={'class':'price_color'})
    
def get_description(soup):
    return soup.find("div", attrs={"id" : "product_description"}).next_sibling.next_sibling

def get_availability(soup):
    available = soup.find('p', attrs={'class':'price_color'}).next_sibling.next_sibling
    return available.find('i').next_sibling

book_data = []
for title_link in title_links: 
    page1 = urlopen(title_link)
    soup = BeautifulSoup(page1, 'html.parser')
    
    title = get_title(soup)
    price = get_price(soup)
    description = get_description(soup)
    availability = get_availability(soup)
    
    book_data += [[title, price, description, availability.strip()]]

In [ ]:
df = pd.DataFrame(book_data)
df.columns = ['title', 'price', 'description', 'availability']
display(df.head())
df.to_csv("booked_scrape.csv")